In [46]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [47]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
## Issues with my environment leading to edited session start
spark=SparkSession.builder.config("spark.driver.host", "localhost").appName('appname').getOrCreate()

In [48]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True, inferSchema=True)


In [49]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [50]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
average_price_query = """
  SELECT 
    date_built AS Year, 
    ROUND(AVG(price), 2) AS Average_Price_4Bedroom
  FROM 
    home_sales
  WHERE 
    bedrooms = 4
  GROUP BY 
    Year
  ORDER BY 
    Year
"""
spark.sql(average_price_query).show()


+----+----------------------+
|Year|Average_Price_4Bedroom|
+----+----------------------+
|2010|             296800.75|
|2011|              302141.9|
|2012|             298233.42|
|2013|             299999.39|
|2014|             299073.89|
|2015|             307908.86|
|2016|             296050.24|
|2017|             296576.69|
+----+----------------------+



In [51]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
average_price_query_3b3ba = """
  SELECT 
    date_built AS Year, 
    ROUND(AVG(price), 2) AS Average_Price_3B_3Ba
  FROM 
    home_sales
  WHERE 
    bedrooms = 3 AND bathrooms = 3
  GROUP BY 
    Year
  ORDER BY 
    Year
"""


spark.sql(average_price_query_3b3ba).show()


+----+--------------------+
|Year|Average_Price_3B_3Ba|
+----+--------------------+
|2010|           292859.62|
|2011|           291117.47|
|2012|           293683.19|
|2013|           295962.27|
|2014|           290852.27|
|2015|            288770.3|
|2016|           290555.07|
|2017|           292676.79|
+----+--------------------+



In [52]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
detailed_avg_price_query = """
  SELECT 
    date_built AS Year, 
    ROUND(AVG(price), 2) AS Average_Price
  FROM 
    home_sales
  WHERE 
    bedrooms = 3 AND 
    bathrooms = 3 AND 
    floors = 2 AND 
    sqft_living >= 2000
  GROUP BY 
    Year
  ORDER BY 
    Year
"""
spark.sql(detailed_avg_price_query).show()


+----+-------------+
|Year|Average_Price|
+----+-------------+
|2010|    285010.22|
|2011|    276553.81|
|2012|    307539.97|
|2013|    303676.79|
|2014|    298264.72|
|2015|    297609.97|
|2016|     293965.1|
|2017|    280317.58|
+----+-------------+



In [53]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()
view_rating_query = """
  SELECT 
    view AS View_Rating, 
    ROUND(AVG(price), 2) AS Average_Price, 
    COUNT(*) AS Num_Homes
  FROM 
    home_sales
  GROUP BY 
    View_Rating
  HAVING 
    AVG(price) >= 350000
  ORDER BY 
    View_Rating DESC
"""
spark.sql(view_rating_query).show()

elapsed_time = time.time() - start_time
print(f"--- {elapsed_time:.2f} seconds ---")

+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.17 seconds ---


In [54]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [55]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [56]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

result = spark.sql("""
    SELECT 
        view AS View_Rating, 
        ROUND(AVG(price), 2) AS Average_Price, 
        COUNT(*) AS Num_Homes
    FROM 
        home_sales
    GROUP BY 
        View_Rating
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        View_Rating DESC
""")

result.show()

elapsed_time = time.time() - start_time
print(f"--- {elapsed_time:.2f} seconds ---")


+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.15 seconds ---


In [57]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [58]:
# 11. Read the formatted parquet data.
partitioned_df=spark.read.parquet('home_sales_partitioned')

In [59]:
# 12. Create a temporary table for the parquet data.
partitioned_df.createOrReplaceTempView('home_sales_p')

In [60]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()
query_results = spark.sql("""
    SELECT 
        view AS View_Rating, 
        ROUND(AVG(price), 2) AS Average_Price, 
        COUNT(*) AS Num_Homes
    FROM 
        home_sales_p
    GROUP BY 
        View_Rating
    HAVING 
        AVG(price) >= 350000
    ORDER BY 
        View_Rating DESC
""")


query_results.show()

execution_time = time.time() - start_time
print(f"--- {execution_time:.2f} seconds ---")

+-----------+-------------+---------+
|View_Rating|Average_Price|Num_Homes|
+-----------+-------------+---------+
|        100|    1026669.5|       28|
|         99|   1061201.42|       31|
|         98|   1053739.33|       33|
|         97|   1129040.15|       27|
|         96|   1017815.92|       36|
|         95|    1054325.6|       45|
|         94|    1033536.2|       50|
|         93|   1026006.06|       33|
|         92|    970402.55|       42|
|         91|   1137372.73|       41|
|         90|   1062654.16|       37|
|         89|   1107839.15|       40|
|         88|   1031719.35|       31|
|         87|    1072285.2|       40|
|         86|   1070444.25|       44|
|         85|   1056336.74|       39|
|         84|   1117233.13|       31|
|         83|   1033965.93|       40|
|         82|    1063498.0|       43|
|         81|   1053472.79|       33|
+-----------+-------------+---------+
only showing top 20 rows

--- 0.28 seconds ---


In [61]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [62]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False